In [1]:
import pandas as pd
import numpy as np
import csv
import re
import string
import nltk

from pythainlp.tokenize import Tokenizer
from pythainlp import word_tokenize
from pythainlp.tokenize import dict_trie

from nltk.corpus import words
from stop_words import get_stop_words

# Function

In [2]:
# Clean product details
def clean_text(msg): 

    msg = str(msg).lower()
    
    regex_na = re.compile("n/a")
    if re.findall(regex_na, msg):
        msg = " "
    
    else:
        msg = re.sub(r"\,", "", str(msg))
        msg = re.sub(r"\–|\~", "-", str(msg))
        msg = re.sub(r"(\+\/\-)|(±)\s?\d+", "", str(msg))
        msg = re.sub(r"(\d+\s?)-(\s?\d+)", r'\1 to \2', str(msg))
        msg = re.sub(r"\(.+\)", "", str(msg))
        msg = re.sub(r"[^a-z0-9ก-๙\.\:]", " ", str(msg))
        msg = re.sub(r"\s+", " ", str(msg))
    # Delete punctuation
   # str_punc = string.punctuation.replace(".","")
   # for c in str_punc:
    #    msg = re.sub(r'\{}'.format(c),'',str(msg))
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

# Clean product name
def clean_prod_name(msg):
    
    msg = str(msg).lower()
    msg = re.sub(r"[^a-z0-9ก-๙\.]", " ", str(msg))
    msg = re.sub(r"\s+", " ", str(msg))
    
    return msg

# Load data

## Product details

In [3]:
def get_data_from_csv(file_name, idx_col):
    
    FILE = './Data/{}.csv'.format(file_name)
    df_1 = pd.read_csv(FILE, index_col=idx_col)
    return df_1

In [4]:
# Load data from csv 

file_name = 'data_jib_1'
file_name_2 = 'data_jib_2'
prod_name = 'productname'
prod_name_2 = 'mistake_get_data'

df_1 = get_data_from_csv(file_name, 0)
df_2 = get_data_from_csv(file_name_2, 0)
prod_name_df = get_data_from_csv(prod_name, 1)
prod_name_df_2 = get_data_from_csv(prod_name_2, 3)

In [5]:
df_1.head()

,P_0Q,P_0A,P_1Q,P_1A,P_2Q,P_2A,P_3Q,P_3A,P_4Q,P_4A,...,P_3882Q,P_3882A,P_3883Q,P_3883A,P_3884Q,P_3884A,P_3885Q,P_3885A,P_3886Q,P_3886A
0,CPU Brand,AMD,CPU Brand,Intel,CPU Brand,AMD,CPU Brand,Intel,CPU Brand,Intel,...,Model,INTEL 1151 CORE I5-9400F 2.90 GHz,Model,INTEL 1151 CORE I5-9600KF 3.70 GHz,Model,AMD AM4 RYZEN 5 3600 3.6GHz,Model,CPU INTEL 1151 CORE I7-9700KF,Model,AMD AM4 RYZEN 7 3700X 3.6GHz
1,CPU Model,AMD Ryzen 5 3500U,CPU Model,Intel Core i5-10210U,CPU Model,AMD Ryzen 5 3500U,CPU Model,Intel Core i5-1035G1,CPU Model,Intel Core i5-9300H,...,CPU Core / Thread,6/6,CPU Core / Thread,6/6,CPU Core / Thread,6/12,CPU Core / Thread,8/8,CPU Core / Thread,8/16
2,Base Clock,2.00 GHz,Base Clock,1.60 GHz,Base Clock,2.10 GHz,Base Clock,1.00 GHz,Base Clock,2.40 GHz,...,Integrated Graphics,-,Integrated Graphics,-,Integrated Graphics,-,Integrated Graphics,-,Integrated Graphics,-
3,Boost Clock,3.60 GHz,Boost Clock,4.20 GHz,Boost Clock,3.70 GHz,Boost Clock,3.60 GHz,Boost Clock,4.10 GHz,...,Model,GIGABYTE B365 M AORUS ELITE,Model,ID COOLING AURAFLOW X 240 RGB,Model,ASROCK B450 STEEL LEGEND,Model,DEEPCOOL CASTLE 280 ARGB,Model,GIGABYTE B450 AORUS ELITE
4,Cache,4 MB,Cache,6 MB,Cache,4 MB,Cache,6 MB,Cache,8 MB,...,Chipset,B365,Model,MSI Z390-A PRO,Chipset,B450,Model,GIGABYTE Z390 AORUS ELITE,Chipset,B450


In [6]:
df_2.head()

,P_101Q,P_101A,P_102Q,P_102A,P_103Q,P_103A,P_104Q,P_104A,P_105Q,P_105A,...,P_4060Q,P_4060A,P_4061Q,P_4061A,P_4062Q,P_4062A,P_4063Q,P_4063A,P_4064Q,P_4064A
0,Detail,Spin Speed : 2200+/-10 RPM,Detail,Dimension : 128.5 x 140 x 168.3 mm<end line>Fa...,Detail,Spin Speed : 2200 RPM<end line>Dimensions : 11...,Detail,Product Name : Wraith Ripper<end line>Product ...,Detail,Max. Rotational Speed (+/- 10%) 1500 RPM<end l...,...,Detail,รูปแบบ : WD DRPP-UPGRADE TO 3 Year<end line>อุ...,Detail,รูปแบบ : DRPP-FULL 3 Year<end line>อุปกรณ์ : L...,Detail,รูปแบบ : DRPP-LITE 1 Year<end line>อุปกรณ์ : L...,Detail,รูปแบบ : DRPP-FULL 1 Year<end line>อุปกรณ์ : L...,Detail,รูปแบบ : DRPP-LITE 3 Year<end line>อุปกรณ์ : L...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_1 = df_1.fillna('-')
df_2 = df_2.fillna('-')

In [8]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 0 to 22
Columns: 6816 entries, P_101Q to P_4064A
dtypes: object(6816)
memory usage: 1.2+ MB


## Product names

In [9]:
prod_name_df.head()

,Unnamed: 0,Product_Name
index,,
0,0,NOTEBOOK (โน้ตบุ๊ค) ASUS VIVOBOOK 15 X512DA-EJ...
1,0,NOTEBOOK (โน้ตบุ๊ค) ACER ASPIRE 3 A315-55G-53C...
2,0,NOTEBOOK (โน้ตบุ๊ค) ASUS VIVOBOOK 15 X512DA-EJ...
3,0,NOTEBOOK (โน้ตบุ๊ค) HP 15S-FQ1012TU (SILVER)
4,0,NOTEBOOK (โน้ตบุ๊ค) DELL INSPIRON 7591-W567015...


In [10]:
prod_name_df['Product_Name'][0]

'NOTEBOOK (โน้ตบุ๊ค) ASUS VIVOBOOK 15 X512DA-EJ139T (TRANSPARENT SILVER)'

In [11]:
prod_name_df_2['Name'][101]

'CPU AIR COOLER (พัดลมซีพียู) DEEPCOOL ICEEDGE MINI FS'

# Extract attributes from dataframe

In [12]:
def extract_attr_df_1(df):
    attr_val_dict = {}
    relations = []

    regex_attr = re.compile("P\_\d+Q")
    regex_val = re.compile("P\_\d+A")
    regex_idx = re.compile("\d+")
    regex_detail = re.compile("Detail")
    
    for idx in df:
        idx_prod = re.findall(regex_idx, idx)[0]
        # Attribute column
        if re.match(regex_attr, idx):
            tmp_list = []
            if not re.match("Detail", df[idx][0]):
                for i,attr in enumerate(df[idx]):
                    test_val = clean_text(df[f"P_{idx_prod}A"][i])
                    if attr != '-' and test_val != '0' and test_val != '' and test_val != '-' and test_val != 'no data':
                        tmp_list.append(clean_text(attr))
                        relations.append(clean_text(attr))
        # Value column        
        elif re.match(regex_val, idx):
            attr_val_dict[idx_prod] = []
            tmp_list_2 = []
            # If attributes and values are pairs
            if not re.match("Detail", df[f"P_{idx_prod}Q"][0]):
                for i,val in enumerate(df[idx]):
                    check_val = clean_text(val)
                    if check_val != '' and check_val != '0' and check_val != '-' and check_val != 'no data':
                        tmp_list_2.append(check_val)
                    
                for i in range(len(tmp_list_2)-1):
                    attr_val_dict[idx_prod].append([tmp_list[i],tmp_list_2[i]])
                    
            # If attributes and values are in 1 cell
            else:
                for v in df[idx]:
                    if v == '-':
                        break
                    # Wrong <end line>
                    if re.findall(r'\:<end line>', v):
                        v = re.sub(r"\:<end line>", "\: ", v)
                    tmp = [str(v).split('<end line>')]
                    # Split attribute and value by ':'
                    attr_val = []
                    for l in tmp:
                        for s in l:
                            if re.findall(':', s):
                                tmp_split = [clean_text(s).split(':')]
                                attr_val.extend(tmp_split)
                    attr_val_dict[idx_prod].extend(attr_val)
                
    return attr_val_dict, relations

In [13]:
df_2['P_110A'][0]

'Product Name :<end line>MasterAir MA410M TUF Gaming Edition<end line>Model :<end line>MAM-T4PN-AFNPC-R1<end line>CPU Socket :<end line>Intel® LGA 2066 / 2011-v3 / 2011 / 1366 / 1156 / 1155 / 1151 / 1150 socket<end line><end line>AMD® AM4 / AM3+ / AM3 / AM2+ / AM2 / FM2+ / FM2 / FM1 socket<end line>Dimensions (LxWxH) :<end line>111.8 x 130.9 x 165.1 mm (4.4 x 5.1 x 6.5 inch)<end line>Heat Sink Material :<end line>4 Heat Pipes / CDC 2.0 / Aluminum Fins<end line>Heat Sink Weight :<end line>820g (1.8 lb.)<end line>Heat Pipe Dimensions :<end line>Ø6 mm<end line>Fan Dimensions :<end line>120 x 120 x 25 mm (4.7 x 4.7 x 1 inch)<end line>Fan Speed :<end line>600 – 1,800 RPM (PWM) ± 10%<end line>Fan Air Flow :<end line>53.38 CFM (Max)<end line>Fan Air Pressure :<end line>1.65mm H2O (Max)<end line>Fan MTTF :<end line>40,000 hours<end line>Fan Noise Level :<end line>6-31 dBA<end line>Fan Connector :<end line>4-Pin (PWM)<end line>Fan Rated Voltage :<end line>12 VDC<end line>Fan Rated Current :<end

In [14]:
%%time
attr_val_1, relations_1 = extract_attr_df_1(df_1)
attr_val_2, relations_2 = extract_attr_df_1(df_2)

Wall time: 3.45 s


In [ ]:
dict(list(attr_val_2.items())[0:10])

In [16]:
dict(list(attr_val_1.items())[0:1])

{'0': [['cpu brand', 'amd'],
  ['cpu model', 'amd ryzen 5 3500u'],
  ['base clock', '2.00 ghz'],
  ['boost clock', '3.60 ghz'],
  ['cache', '4 mb'],
  ['graphic chip', 'amd radeon vega 8'],
  ['brand', 'amd'],
  ['display size', '15.60'],
  ['display resolution', '1920 x 1080'],
  ['display type', 'full hd'],
  ['touchscreen', 'no'],
  ['ram', '8 gb'],
  ['ram type', 'ddr4'],
  ['hdd 1 capacity', '512 gb'],
  ['hdd 1 type', 'm.2 ssd'],
  ['optical disk drive', 'no'],
  ['wi fi', 'wi fi 5'],
  ['bluetooth', '4.1'],
  ['sim card', 'no'],
  ['os', 'windows 10'],
  ['web camera', 'yes'],
  ['battery', '2 cell'],
  ['warranty', '2 years'],
  ['usb 2.0', 'yes'],
  ['usb 3.0', 'no'],
  ['usb 3.1', 'yes'],
  ['usb 3.2', 'no'],
  ['usb c', 'yes'],
  ['displayport', 'no'],
  ['hdmi', 'yes'],
  ['d sub', 'no'],
  ['thunderbolt', 'no'],
  ['card reader', 'microsd'],
  ['finger print reader', 'no'],
  ['weight', '1.6 kg'],
  ['color', 'silver']]}

# Keep attributes and values in triples form

In [17]:
def create_triples_df_1(attr_val):
    triples = []
    #regex_none = re.compile("\s|0|^$")
    for key,val in attr_val.items():
        for v in val:
            if len(v) > 1:
                tmp_triples = []
                tmp_triples.extend([clean_prod_name(prod_name_df['Product_Name'][int(key)]), v[0], v[1]])
                triples.append(tmp_triples)
    return triples


def create_triples_df_2(attr_val):
    triples = []
    #regex_none = re.compile("\s|0|^$")
    for key,val in attr_val.items():
        for v in val:
            if len(v) > 1:
                tmp_triples = []
                tmp_triples.extend([clean_prod_name(prod_name_df_2['Name'][int(key)]), v[0], v[1]])
                triples.append(tmp_triples)
    return triples

In [18]:
%%time
triples_df_1 = create_triples_df_1(attr_val_1)
triples_df_2 = create_triples_df_2(attr_val_2)

Wall time: 734 ms


In [21]:
triples_df_1

[['notebook โน้ตบุ๊ค asus vivobook 15 x512da ej139t transparent silver ',
  'cpu brand',
  'amd'],
 ['notebook โน้ตบุ๊ค asus vivobook 15 x512da ej139t transparent silver ',
  'cpu model',
  'amd ryzen 5 3500u'],
 ['notebook โน้ตบุ๊ค asus vivobook 15 x512da ej139t transparent silver ',
  'base clock',
  '2.00 ghz'],
 ['notebook โน้ตบุ๊ค asus vivobook 15 x512da ej139t transparent silver ',
  'boost clock',
  '3.60 ghz'],
 ['notebook โน้ตบุ๊ค asus vivobook 15 x512da ej139t transparent silver ',
  'cache',
  '4 mb'],
 ['notebook โน้ตบุ๊ค asus vivobook 15 x512da ej139t transparent silver ',
  'graphic chip',
  'amd radeon vega 8'],
 ['notebook โน้ตบุ๊ค asus vivobook 15 x512da ej139t transparent silver ',
  'brand',
  'amd'],
 ['notebook โน้ตบุ๊ค asus vivobook 15 x512da ej139t transparent silver ',
  'display size',
  '15.60'],
 ['notebook โน้ตบุ๊ค asus vivobook 15 x512da ej139t transparent silver ',
  'display resolution',
  '1920 x 1080'],
 ['notebook โน้ตบุ๊ค asus vivobook 15 x512da ej139

In [19]:
def save_to_txt(triples, file_name):
    # Write the triples to text file
    with open('./Data/triples_{}.txt'.format(file_name), 'w', encoding='utf-8') as f:
        for _list in triples:
            for _string in _list:
                #f.seek(0)
                f.write(str(_string) + '\t')
            f.write('\n')
    return

In [20]:
save_to_txt(triples_df_1, file_name)
save_to_txt(triples_df_2, file_name_2)